In [3]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [4]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [5]:
world_topo = data.world_110m()

## Marcas de Formas Geográficas

Para visualizar dados geográficos, o Altair fornece o tipo de marca `geoshape`. Para criar um mapa básico, podemos criar uma marca `geoshape` e passar nossos dados TopoJSON, que são então desempacotados em recursos GeoJSON, um para cada país do mundo:


In [6]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplica uma cor azul padrão e usa uma projeção de mapa padrão (`equalEarth`). Podemos personalizar as cores e a largura das linas de fronteiras usando propriedades padrão da marca. Usando o método `project`, também podemos adicionar nossa própria projeção de mapa:

In [7]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados se encaixem na largura e na altura do gráfico. Também podemos especificar parâmetros de projeção, tal como `scale` (nível de zoom) e `translate` (panorâmica), para personalizar as configurações de projeção. Aqui ajustamos os parâmetros `scale` e `translate` para focar na Europa:

In [8]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_Observe como a resolução de 110m dos dados se torna aparente nessa escala. Para ver contornos da costa e fronteiras mais detalhados, precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e `translate` para focar o mapa em outras regiões!_



Até agora, nosso mapa mostra apenas países. Usando o operador `layer`, podemos combinar vários elementos de mapa. O Altair inclui _geradores de dados_ que podemos usar para criar dados para camadas de mapa adicionais: 

- O gerador de esfera (`{'sphere': True}`) fornece uma representação geojson da esfera completa da terra. Podemos criar uma marca de `geoshape` adicional que preencha a forma da Terra como uma camada de fundo.
- O gerador de gratícula (`{'graticule': ...}`) cria um recurso Geojson representando uma gratícula: uma grade formada por linhas de latitude e longitude. A gratícula padrão possui meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, existem meridianos a cada 90°. Essas configurações podem ser personalizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos dispor em camadas as marcas esfera, gratícula e país em uma especificação de mapa reutilizável:

In [9]:
map = alt.layer(
    # Use a esfera da terra como a camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # Adicione uma gratícula para linhas de referência geográfica
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e então os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Podemos estender o mapa com uma projeção desejada e desenhar o resultado. Aqui aplicamos uma [Projeção natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada de _esfera_ fornece o fundo azul claro; A camada de gratícula fornece as linhas de referência geográficas brancas.

In [10]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## Mapas de Pontos

Além dos dados _geométricos_ fornecidos pelos arquivos GeoJSON ou TopoJSON, muitos conjuntos de dados tabulares incluem informações geográficas na forma de campos para coordenadas de `longitude` e `latitude`, ou referências a regiões geográficas, como nomes de países, nomes de estados, códigos postais, _etc._, que pode ser mapeado para coordenadas usando um [serviço de geocodificação](https://en.wikipedia.org/wiki/Geocoding). Em alguns casos, os dados de localização são ricos o suficiente para que possamos ver padrões significativos projetados apenas pelos pontos de dados &mdash; nenhum mapa base é necessário! 

Vejamos um conjunto de dados de códigos postais de 5 dígitos nos Estados Unidos, incluindo coordenadas de `longitude` e `latitude` para cada agência dos correios, além de um campo `zip_code`.

In [11]:
zipcodes = data.zipcodes.url
zipcodes

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/zipcodes.csv'

Podemos visualizar a localização de cada agência dos correios usando uma pequena marca `square` (1 pixel). No entanto, para definir as posições, _não_ usamos canais `x` e `y`. Por que?

Embora as projeções cartográficas mapeie coordenadas (longitude, latitude) para as coordenadas coordenadas (`x`,`y`), elas podem fazê-lo de maneiras arbitrárias. Não há garantia, por exemplo, que `longitude` → `x` e `latitude` → `y`! 
Em vez disso, o Altair inclui canais codificadores especiais de `longitude` e `latitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados para coordenadas de `longitude` e `latitude`, e em seguida aplica uma projeção para mapear essas coordenadas para posições (`x`, `y`).

In [12]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplique o campo chamado 'longitude' no canal longitude
    latitude='latitude:Q'    # aplique o campo chamado 'latitude' no canal latitude
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Ao plotar apenas os códigos postais, podemos ver o contorno dos Estados Unidos e discernir padrões significativos na densidade de agências dos correios, sem a necessidade de um mapa base ou elementos de referência adicionais!_

Usamos a projeção `albersUsa`, que faz algumas adaptações na geometria real da Terra, com versões escalonadas do Alasca e do Havai no canto inferior esquerdo. Como não especificamos parâmetros de projeção `scale` ou `translate`, o Altair os define automaticamente para se ajustar aos dados visualizados.

Agora podemos seguir em frente e fazer mais perguntas sobre nosso dataset. Por exemplo, existe algum padrão na alocação dos códigos postais? Para avaliar essa questão, podemos adicionar uma codificação de cor com base no primeiro dígito do código postal. Primeiro, adicionamos uma transformação `calculate` para extrair o primeiro dígito e codificamos o resultado usando o canal de cor.

In [13]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=alt.Color('digit:N',title='dígito')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para dar zoom em um dígito específico, adicione uma transformação de filtro para limitar os dados exibidos! Tente adicionar uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar por um único dígito e atualizar dinamicamente o mapa. E lembre-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar os valores dos dígitos!_

(Este exemplo é inspirado pela clássica visualização [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)

Podemos ainda nos perguntar o que a _sequência_ dos códigos postais pode indicar. Uma forma de explorar essa questão é conectar cada código postal consecutivo usando uma marca `line`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:

In [14]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Agora podemos ver como os códigos postais se agrupam em áreas menores, indicando uma alocação hierárquica dos códigos por local, mas com uma variação notável dentro dos grupos locais._

Se você estava prestando atenção aos nossos mapas anteriores, talvez pode ter notado que há códigos postais sendo plotados no canto superior esquerdo! Esses correspondem a locais como Porto Rico ou Samoa Americana, que contêm códigos postais dos EUA, mas são mapeados para coordenadas `null` (`0`, `0`) pela projeção `albersUsa`. Além disso, Alasca e Havai podem complicar nossa visão dos segmentos de linha conectados. Em resposta, o código acima inclui um filtro adicional que remove pontos fora dos intervalos de `longitude` e `latitude` escolhidos.

_Remova o filtro acima para ver o que acontece!_

## Mapas de Símbolos

Agora, vamos combinar um mapa base e os dados plotados como camadas separadas. Vamos examinar a rede de voos comerciais dos EUA, considerando tanto os aeroportos quanto as rotas de voo. Para isso, precisaremos de três conjuntos de dados. 
Para o nosso mapa base, usaremos um arquivo TopoJSON dos Estados Unidos com resolução de 10m, contendo recursos para `states` ou `counties`:

In [15]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

Para os aeroportos, usaremos um dataset com campos para as coordenadas de `longitude` e `latitude` de cada aeroporto, bem como o código do aeroporto `iata` &mdash; por exemplo, `'SEA'` para [Seattle-Tacoma International Airport](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).

In [16]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

Por fim, utilizaremos um dataset de rotas de voo, que contém os campos `origin` e `destination` com os códigos IATA dos aeroportos correspondentes:

In [17]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

Vamos começar criando um mapa base usando a projeção `albersUsa` e adicionar uma camada que traça marcas `circle` para cada aeroporto:

In [18]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_São muitos aeroportos! Obviamente, nem todos são grandes hubs._

Semelhante ao nosso dataset de códigos postais, os dados dos aeroportos incluem pontos que estão fora dos Estados Unidos continentais. Então, novamente vemos pontos no canto superior esquerdo. Talvez seja necessário filtrar esses pontos, mas, para isso, precisamos saber mais sobre eles.

_Atualize a projeção do mapa acima para `albers` &ndash; contornando o comportamento peculiar do `albersUsa` &ndash; para que as localizações reais desses pontos adicionais sejam reveladas!_

Agora, em vez de mostrar todos os aeroportos de maneira indistinta, vamos identificar os grandes hubs considerando o número total de rotas que se originam em cada aeroporto. Usaremos o dataset `routes` como nossa principal fonte de dados: ele contém uma lista de rotas de voo que podemos agregar para contar o número de rotas para cada aeroporto de `origin`.

No entanto, o dataset `routes` não inclui as _localizações_ dos aeroportos! Para complementar os dados de `routes` com as localizações, precisamos de uma nova transformação de dados: `lookup`. A transformação `lookup` pega o valor de um campo em um dataset principal e o usa como uma _chave_ para procurar informações relacionadas em outra tabela. Neste caso, queremos comparar o código do aeroporto de `origin` no nosso dataset `routes` com o campo `iata` do dataset `airports` e, em seguida, extrair os campos correspondentes de `latitude` e `longitude`.

In [19]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Quais aeroportos dos EUA têm o maior número de rotas de saída?_

Agora que podemos ver os aeroportos, talvez queiramos interagir com eles para entender melhor a estrutura da rede de tráfego aéreo. Podemos adicionar uma camada de marca `rule` para representar os caminhos de aeroportos de `origin` para aeroportos de `destination`, o que requer duas transformações `lookup` para recuperar as coordenadas de cada ponto final. Além disso, podemos usar uma seleção `point` para filtrar essas rotas, de modo que apenas as rotas que se originam no aeroporto atualmente selecionado sejam exibidas.

_Partindo do mapa estático acima, você consegue construir uma versão interativa? Fique à vontade para pular o código abaixo para interagir com o mapa e pensar em como poderia construí-lo por conta própria!_


In [24]:
# Seleção interativa para o aeroporto de origem 
# Seleciona o aeroporto mais próximo ao cursor do mouse 
origin = alt.selection_point(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# Referência de dados compartilhados para transformações de pesquisa
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # mapa base dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas de rota do aeroporto de origem selecionado para aeroportos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtro para origem selecionada apenas 
    ).transform_lookup(
        lookup='origin', from_=foreign # origem lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # destino lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # Tamanho dos Aeroportos por número de rotas de saída  
    # 1. agrega o dataset flights-airport 
    # 2. pesquisa os dados de localização do dataset dos aeroportos 
    # 3. Remove Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_params(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # círculos menores no topo
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Passe o mouse sobre o mapa para explorar a rede de voos!_